In [ ]:
import metadrive
from metadrive import MetaDriveEnv

In [ ]:
class ModifiedMetadriveEnv(MetaDriveEnv):
    def setup_engine(self):
        self.engine.register_manager("agent_manager", self.agent_manager)
        self.engine.accept("b", self.switch_to_top_down_view)
        self.engine.accept("q", self.switch_to_third_person_view)
        self.engine.accept("]", self.next_seed_reset)
        self.engine.accept("[", self.last_seed_reset)
        self.engine.accept("p", self.stop)
        from metadrive.manager.traffic_manager import PGTrafficManager
        from metadrive.manager.pg_map_manager import PGMapManager
        from metadrive.manager.object_manager import TrafficObjectManager
        self.engine.register_manager("map_manager", PGMapManager())
        self.engine.register_manager("traffic_manager", PGTrafficManager())